In [1]:
from pathlib import Path
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy
from tqdm import tqdm

## Define brain, find trace data folder

In [2]:
#specify brain1 or brain2 below
brain = "brain1"

root_dir = Path(os.path.abspath('')).parents[1]
experiment_dir = os.path.join(root_dir, "axon_geometry")
data_dir = os.path.join(experiment_dir, "data", brain)
segments_swc_dir = os.path.join(data_dir, "segments_swc")
trace_data_dir = os.path.join(data_dir, "trace_data")
trace_data_dir = os.path.join(trace_data_dir, "14")
print(f"Directory where swcs reside: {segments_swc_dir}")

Directory where swcs reside: /Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/axon_geometry/data/brain1/segments_swc


## Read trace data

In [3]:
max_id = 300


def classify_height(row):
    height = row["height"]
    if height <= 2:
        return height
    else:
        return 3

def numerical_class(row):
    _class = row["class"]
    if _class == "axon":
        return 0
    if _class == "collateral":
        return 1
    if _class == "terminal":
        return 2

df_path = os.path.join(trace_data_dir, "df.csv")
if os.path.exists(df_path):
    df = pd.read_csv(df_path)
else:
    df = pd.DataFrame(columns=["seg_id", "class", "height", "log_seg_length", "measure", "value", "log_value"])
    for i in np.arange(0, max_id):
        i = int(i)
        trace_data_path = os.path.join(trace_data_dir, "{}.npy".format(i))
        if os.path.exists(trace_data_path) is True:
            trace_data = np.load(trace_data_path, allow_pickle=True)
            print("Loaded segment {}".format(i))

            for node in trace_data:
                seg_length = node["seg_length"]
                height = node["height"]
                _class = node["class"]
                mean_curvature = node["mean_curvature"]
                mean_torsion = node["mean_torsion"]
                
                log_seg_length = np.log10(seg_length)

                log_mean_curvature = np.log10(mean_curvature)
                df = df.append({"seg_id": i, "height": height, "class": _class, "log_seg_length": log_seg_length, "measure": "curvature", "value": mean_curvature, "log_value": log_mean_curvature}, ignore_index=True)

                log_mean_torsion = np.log10(mean_torsion)
                df = df.append({"seg_id": i, "height": height, "class": _class, "log_seg_length": log_seg_length, "measure": "torsion", "value": mean_torsion, "log_value": log_mean_torsion}, ignore_index=True)
    df.to_csv(df_path)
df["class"] = df.apply(numerical_class, axis=1)
df["height_class"] = df.apply(classify_height, axis=1)

# GLM Permutation Test
Using Freedman and Lane 1983 method as described in Winkler et. al. 2014 Permutation inference for the general linear model


In [5]:
n_perms = 10000

measures = ["curvature", "torsion"]

classes = {0: "primary", 1: "collateral", 2: "terminal"}
n = len(classes)
matrix_pairs = np.triu(np.ones((n, n)), k=1)
(coord_pairs_x, coord_pairs_y) = np.where(matrix_pairs == 1)

for measure in measures:
    for class_1, class_2 in zip(coord_pairs_x, coord_pairs_y):
        Y = []
        segment_numbers = []
        X_class = []
        neuron_id = 0
        #collect data from all neurons
        for i in np.arange(0, max_id):
            sample_query = df.loc[(df['seg_id'] == i) & ((df['class'] == class_1) | (df['class'] == class_2)) & (df['measure'] == measure)]
            num_segments = len(sample_query.index)
            if num_segments > 0:
                Y.append(sample_query["value"].to_numpy())
                segment_numbers.append(num_segments)
                neuron_id += 1
                X_class.append(sample_query["class"].to_numpy())

        # setup Y and X matrices
        Y = np.concatenate(Y)
        X_class = np.concatenate(X_class)
        X = np.zeros((Y.shape[0], neuron_id+1))
        # segment class covariate
        X[X_class == class_2,0] = 1

        cs = np.concatenate([[0],np.cumsum(segment_numbers)])
        cumulative_index = 0

        for neuron in range(1, neuron_id):
            X[cs[neuron-1]:cs[neuron],neuron+1] = 1
        X_null = X[:,1:]

        # Below are the steps for the permutation procedure as described in Winkler et. al.
        # Step 1
        X_pinv = np.linalg.pinv(X)
        beta_hat = X_pinv @ Y
        T0 = beta_hat[0]

        # Step 2
        X_null_pinv = np.linalg.pinv(X_null)
        beta_null_hat = X_null_pinv @ Y
        resid_null = Y - X_null @ beta_null_hat

        Tjs = np.zeros((n_perms+1))
        for _iter in tqdm(range(n_perms)):
            # Step 3
            for neuron in range(1, neuron_id):
                resid_null[cs[neuron-1]:cs[neuron]] = np.random.permutation(resid_null[cs[neuron-1]:cs[neuron]])
            Yj = np.random.permutation(resid_null) + X_null @ beta_null_hat

            # Step 4
            beta_hatj = X_pinv @ Yj
            Tjs[_iter] = beta_hatj[0]
        Tjs[-1] = T0
        
        # Step 6
        upper_pval = np.sum(Tjs>=T0)/len(Tjs)
        lower_pval = np.sum(Tjs<=T0)/len(Tjs)
        if  upper_pval < 0.05/6:
            print(f"{classes[class_2]} > {classes[class_1]} in {measure}")
            print(f"p-val was: {upper_pval}")
        elif lower_pval < 0.05/6:
            print(f"{classes[class_2]} < {classes[class_1]} in {measure}")
            print(f"p-val was: {lower_pval}")
        else:
            print(f"{classes[class_2]} ~= {classes[class_1]} in {measure}")
            print(f"upper p-val was: {upper_pval}, lower was {lower_pval}")



100%|██████████| 10000/10000 [00:26<00:00, 381.57it/s]
collateral > primary in curvature
p-val was: 9.999000099990002e-05
100%|██████████| 10000/10000 [00:49<00:00, 202.80it/s]
terminal > primary in curvature
p-val was: 9.999000099990002e-05
100%|██████████| 10000/10000 [01:06<00:00, 151.19it/s]
terminal < collateral in curvature
p-val was: 9.999000099990002e-05
100%|██████████| 10000/10000 [00:26<00:00, 381.63it/s]
collateral > primary in torsion
p-val was: 9.999000099990002e-05
100%|██████████| 10000/10000 [00:52<00:00, 192.16it/s]
terminal ~= primary in torsion
upper p-val was: 0.0570942905709429, lower was 0.943005699430057
100%|██████████| 10000/10000 [01:05<00:00, 151.65it/s]terminal < collateral in torsion
p-val was: 9.999000099990002e-05

